In [1]:
import gc
import shutil
import tempfile
from pathlib import Path
from getpass import getuser

import dask.array as dsa
import numpy as np
import pandas as pd
import pytest
import xarray as xr
from dask import delayed, persist
from dask.distributed import as_completed

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path("/scratch") / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(
    n_workers=4, threads_per_worker=1, scratch_dir=scratch_dir / "clients",
    memory_limit='512MB'
)  # Specify temporary scratch directory for dask to use

/home/b/b382615/opt/anaconda3/envs/py310/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43015 instead
  warnings.warn(


Hostname: l20543
Forward Port: l20543:43015
Dashboard Link: localhost:43015/status


In [4]:
test_data_path = "/home/b/b382615/opt/marEx/tests/data/sst_gridded.zarr"
ds = xr.open_zarr(str(test_data_path), chunks={}).persist()
sst_data = ds.to.isel(time=slice(0,1400))

# Define standard dimensions for gridded data
dimensions = {"time": "time", "xdim": "lon", "ydim": "lat"}

sst_data

<xarray.DataArray 'to' (time: 1400, lat: 20, lon: 40)> Size: 4MB
dask.array<getitem, shape=(1400, 20, 40), dtype=float32, chunksize=(30, 20, 40), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 80B 35.12 35.38 35.62 35.88 ... 39.38 39.62 39.88
  * lon      (lon) float32 160B -39.88 -39.62 -39.38 ... -30.62 -30.38 -30.12
  * time     (time) datetime64[ns] 11kB 1982-01-01T12:00:00 ... 1985-10-31T12...
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [5]:
da_new = sst_data.copy(deep=True)
da_new


<xarray.DataArray 'to' (time: 1400, lat: 20, lon: 40)> Size: 4MB
dask.array<getitem, shape=(1400, 20, 40), dtype=float32, chunksize=(30, 20, 40), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 80B 35.12 35.38 35.62 35.88 ... 39.38 39.62 39.88
  * lon      (lon) float32 160B -39.88 -39.62 -39.38 ... -30.62 -30.38 -30.12
  * time     (time) datetime64[ns] 11kB 1982-01-01T12:00:00 ... 1985-10-31T12...
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [6]:
da_re = xr.DataArray(
    da_new.values,
    dims=['T', 'y', 'x'],
    coords={
        'Time': ('T', da_new.time.values),
        'latitude': ('y', da_new.lat.values),
        'longitude': ('x', da_new.lon.values)
    },
    attrs=da_new.attrs
)
da_re

<xarray.DataArray (T: 1400, y: 20, x: 40)> Size: 4MB
array([[[292.1016 , 292.3412 , 292.4408 , ..., 291.42242, 291.47842,
         291.5516 ],
        [292.07   , 292.2564 , 292.3564 , ..., 291.3132 , 291.3096 ,
         291.3324 ],
        [291.9672 , 292.0008 , 292.1488 , ..., 291.25122, 291.14798,
         291.06476],
        ...,
        [290.29843, 290.318  , 290.3952 , ..., 290.0568 , 290.03918,
         289.98355],
        [290.31042, 290.3288 , 290.3916 , ..., 290.07678, 289.9712 ,
         289.8772 ],
        [290.34158, 290.35   , 290.3844 , ..., 289.91037, 289.77118,
         289.65958]],

       [[291.96198, 292.1376 , 292.23477, ..., 291.3844 , 291.386  ,
         291.46918],
        [291.9252 , 292.08557, 292.164  , ..., 291.2788 , 291.2472 ,
         291.26602],
        [291.8372 , 291.8732 , 291.99878, ..., 291.1908 , 291.11957,
         291.03638],
...
        [292.816  , 292.9376 , 292.8928 , ..., 291.42477, 291.65198,
         291.9992 ],
        [292.57202, 292.656  , 292.75   , ..., 291.49   , 291.5888 ,
         291.9624 ],
        [292.35278, 292.3284 , 292.37482, ..., 291.26718, 291.3944 ,
         291.814  ]],

       [[293.7832 , 293.9332 , 294.1528 , ..., 294.5384 , 294.7456 ,
         294.9152 ],
        [293.80118, 293.98077, 294.11038, ..., 294.4044 , 294.5204 ,
         294.91925],
        [293.906  , 293.86722, 293.856  , ..., 294.46997, 294.4168 ,
         294.692  ],
        ...,
        [292.20078, 292.22763, 292.3364 , ..., 291.42038, 291.5264 ,
         291.79642],
        [291.71237, 291.76718, 292.05722, ..., 291.4568 , 291.40277,
         291.64398],
        [292.0664 , 291.90762, 291.98038, ..., 291.13278, 291.1688 ,
         291.43564]]], shape=(1400, 20, 40), dtype=float32)
Coordinates:
    Time       (T) datetime64[ns] 11kB 1982-01-01T12:00:00 ... 1985-10-31T12:...
    latitude   (y) float32 80B 35.12 35.38 35.62 35.88 ... 39.38 39.62 39.88
    longitude  (x) float32 160B -39.88 -39.62 -39.38 ... -30.62 -30.38 -30.12
Dimensions without coordinates: T, y, x
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [7]:
da_re.isel(T=10)

<xarray.DataArray (y: 20, x: 40)> Size: 3kB
array([[291.5084 , 291.6496 , 291.7948 , 291.9328 , 291.862  , 291.8064 ,
        291.7752 , 291.8632 , 291.57877, 291.4188 , 291.2884 , 291.1608 ,
        291.19397, 291.2524 , 291.21878, 291.12558, 290.9576 , 291.01602,
        291.4904 , 291.48758, 291.9868 , 292.5416 , 292.55   , 292.50922,
        292.51038, 292.23322, 291.83716, 291.3868 , 291.24036, 291.2676 ,
        291.35477, 291.368  , 291.44397, 291.1868 , 290.68802, 290.3532 ,
        290.3472 , 290.1348 , 290.0716 , 290.35718],
       [291.5684 , 291.67358, 291.6956 , 291.69757, 291.8084 , 291.7992 ,
        291.6776 , 291.8108 , 291.486  , 291.288  , 291.3408 , 291.3476 ,
        291.1992 , 291.18878, 291.17798, 290.992  , 290.77277, 290.9992 ,
        291.7456 , 291.76962, 291.63757, 292.2056 , 292.5324 , 292.10077,
        291.6636 , 291.23962, 291.136  , 291.1764 , 291.27158, 291.1092 ,
        291.126  , 291.2348 , 291.2588 , 290.9868 , 290.6132 , 290.522  ,
        290.49957, 290.2804 , 290.2052 , 290.5592 ],
       [291.62637, 291.74237, 291.73117, 291.66364, 291.6832 , 291.65598,
        291.6152 , 291.58636, 291.3212 , 291.21198, 291.2176 , 290.9956 ,
        290.7652 , 291.0944 , 291.1964 , 290.92722, 290.9444 , 291.1928 ,
        291.67477, 291.9264 , 291.5276 , 291.48596, 291.95438, 291.7644 ,
        291.13278, 290.826  , 290.97598, 291.232  , 291.32516, 291.0148 ,
        290.724  , 290.71237, 290.736  , 290.86078, 290.87637, 290.8504 ,
...
        289.952  , 289.7904 , 289.86038, 290.09964, 290.0628 , 289.9736 ,
        289.9524 , 289.8868 , 289.8336 , 289.6576 , 289.24118, 289.40482,
        289.70276, 289.7872 , 289.7128 , 289.59882, 289.586  , 289.698  ,
        289.906  , 290.07758, 289.9868 , 289.9336 , 289.85   , 289.8892 ,
        289.80362, 289.86038, 290.0192 , 289.95038, 289.8396 , 289.79956,
        289.7956 , 289.962  , 290.1212 , 289.90118],
       [290.346  , 290.3556 , 290.29358, 290.17358, 289.9544 , 289.8204 ,
        289.8548 , 289.91718, 289.96237, 289.95642, 289.796  , 289.7204 ,
        289.6432 , 289.59763, 289.65762, 289.65118, 289.68277, 289.6856 ,
        289.5972 , 289.54758, 289.4732 , 289.4368 , 289.5368 , 289.66202,
        289.7868 , 289.8684 , 289.94   , 289.88998, 289.684  , 289.81158,
        289.9148 , 289.8568 , 289.8212 , 289.6708 , 289.5564 , 289.5972 ,
        289.7052 , 289.8504 , 289.8728 , 289.678  ],
       [290.2852 , 290.27078, 290.1884 , 289.8916 , 289.65   , 289.57278,
        289.5228 , 289.56238, 289.6    , 289.5336 , 289.4652 , 289.44   ,
        289.4408 , 289.58878, 289.72638, 289.6916 , 289.70956, 289.6908 ,
        289.578  , 289.5196 , 289.46838, 289.42917, 289.496  , 289.53558,
        289.5156 , 289.5064 , 289.66083, 289.59   , 289.36   , 289.4176 ,
        289.64722, 289.6788 , 289.6388 , 289.51917, 289.4276 , 289.488  ,
        289.55237, 289.5832 , 289.56198, 289.48477]], dtype=float32)
Coordinates:
    Time       datetime64[ns] 8B 1982-01-11T12:00:00
    latitude   (y) float32 80B 35.12 35.38 35.62 35.88 ... 39.38 39.62 39.88
    longitude  (x) float32 160B -39.88 -39.62 -39.38 ... -30.62 -30.38 -30.12
Dimensions without coordinates: y, x
Attributes:
    comment:         OSTIA foundation SST
    long_name:      analysed sea surface temperature
    reference:      C.J. Donlon, M. Martin,J.D. Stark, J. Roberts-Jones, E. F...
    source:         AMSR2-REMSS-L2P-v2.0, AMSRE-REMSS-L2P-v2.0, TMI-REMSS-L2P...
    standard_name:  sea_surface_foundation_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [8]:
da_re.isel(Time=10)

ValueError: Dimensions {'Time'} do not exist. Expected one or more of ('T', 'y', 'x')

In [9]:
da_re["Time"]

<xarray.DataArray 'Time' (T: 1400)> Size: 11kB
array(['1982-01-01T12:00:00.000000000', '1982-01-02T12:00:00.000000000',
       '1982-01-03T12:00:00.000000000', ...,
       '1985-10-29T12:00:00.000000000', '1985-10-30T12:00:00.000000000',
       '1985-10-31T12:00:00.000000000'],
      shape=(1400,), dtype='datetime64[ns]')
Coordinates:
    Time     (T) datetime64[ns] 11kB 1982-01-01T12:00:00 ... 1985-10-31T12:00:00
Dimensions without coordinates: T

In [10]:
da_re["T"]

<xarray.DataArray 'T' (T: 1400)> Size: 11kB
array([   0,    1,    2, ..., 1397, 1398, 1399], shape=(1400,))
Coordinates:
    Time     (T) datetime64[ns] 11kB 1982-01-01T12:00:00 ... 1985-10-31T12:00:00
Dimensions without coordinates: T

In [ ]:
extremes_ds = marEx.preprocess_data(
            sst_data,
            method_anomaly="shifting_baseline",
            method_extreme="hobday_extreme",
            threshold_percentile=85,  # Lower threshold for test data
            window_year_baseline=1,  # Reduced for test data duration
            smooth_days_baseline=11,  # Reduced smoothing window
            window_days_hobday=5,  # Reduced hobday window
            dimensions=dimensions,
            dask_chunks={"time": 15},
        )